In [ ]:
import pandas as pd
pd.set_option("display.max_rows", 8)
pd.set_option("display.max_columns", None)
import tkinter as tk
from tkinter import filedialog

In [ ]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

In [ ]:
df = pd.read_excel(file_path, sheet_name="出貨明細")

In [ ]:
iris = df[["BG", "Subcategory", "Group", "狀態", "預交年份", "預交月份", "負責業務", "品名", "本國幣別NTD"]]

In [ ]:
iris = iris[(iris["BG"] == "RF") & (iris["狀態"].str.contains("出")) & (iris["預交年份"] == 2021) & (iris["預交月份"].isin(["February", "March"]))]

In [ ]:
iris["Subcategory"] = iris.apply(lambda x: "BU2" if "RFDPA" in x["品名"] else "BU1", axis=1)

In [ ]:
result = iris.pivot_table(index=["負責業務", "Subcategory"], values="本國幣別NTD", columns="預交月份", aggfunc="sum").reset_index()

In [ ]:
test = iris.groupby(["Subcategory", "負責業務", "預交月份"])[["本國幣別NTD"]].sum().unstack("預交月份").reset_index()

In [ ]:
result = iris.pivot_table(index=["負責業務", "Subcategory"], values="本國幣別NTD", aggfunc="sum", margins=True, margins_name="Total").reset_index()

In [ ]:
final = pd.DataFrame(result.to_records())

In [ ]:
final = final.drop("index", axis=1)

In [ ]:
final = final.sort_values(["Subcategory", "February", "March"], ascending=[True, False, False])
final.style.format({"February":"{0:,.0f}", "March":"{0:,.0f}"})

In [ ]:
result.style.format({"本國幣別NTD":"{0:,.0f}"}).highlight_max(color="lightgreen").highlight_min(color="red")